# Create a sphere using PyVista

This notebook creates a sphere using PyVista. 2 different methods are used:

1. Use PyVista's Delaunay triangulation method to create a sphere.
2. Construct a sphere from scratch by generating arrays for vertices, triangles and vertex normals.


In [ ]:
import numpy as np
import pyvista as pv
from scipy.spatial import ConvexHull

## 1 - Generate a cube using PyVista Delaunay triangulation.


### 1.1 - Vertices

We want to generate an array of vertices with the correct shape.


Make arrays of $r$, $\theta$, $\phi$ values.


In [ ]:
r = np.array([1])
theta = np.linspace(0, np.pi, 10)
phi = np.linspace(0, 2 * np.pi, 20)

Convert from spherical coordinates to Cartesian coordinates.


In [ ]:
r_v, theta_v, phi_v = np.meshgrid(np.array([1]), theta, phi)

x_v = r_v * np.sin(theta_v) * np.cos(phi_v)
y_v = r_v * np.sin(theta_v) * np.sin(phi_v)
z_v = r_v * np.cos(theta_v)

Combine the $x$, $y$, $z$ values into an array, `vertices`, with the correct shape. `vertices` should have shape ($NM$, 3), where $N$, $M$ is the number of $\theta$, $\phi$ values.


In [ ]:
vertices = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

vertices.shape

### 1.2 - PyVista mesh

We want to create a PyVista mesh.


Create and plot the PyVista mesh. This mesh only has vertices (no faces).


In [ ]:
mesh = pv.PolyData(vertices)
mesh.plot()

### 1.3 - Connected surface

We want to create a connected surface.


Create and plot a connected surface, using Delaunay triangulation.


In [ ]:
surface = mesh.delaunay_3d()
surface.plot(show_edges=True)

## 2 - Generate a sphere from scratch.


### 2.1 - Vertices

We want to generate an array of vertices with the correct shape.


Make arrays of $r$, $\theta$, $\phi$ values.


In [ ]:
r = np.array([1])
theta = np.linspace(0, np.pi, 10)
phi = np.linspace(0, 2 * np.pi, 20)

Convert from spherical coordinates to Cartesian coordinates.


In [ ]:
r_v, theta_v, phi_v = np.meshgrid(np.array([1]), theta, phi)

x_v = r_v * np.sin(theta_v) * np.cos(phi_v)
y_v = r_v * np.sin(theta_v) * np.sin(phi_v)
z_v = r_v * np.cos(theta_v)

Combine the $x$, $y$, $z$ values into an array, `vertices`, with the correct shape. `vertices` should have shape ($NM$, 3), where $N$, $M$ is the number of $\theta$, $\phi$ values.


In [ ]:
vertices = np.c_[x_v.reshape(-1), y_v.reshape(-1), z_v.reshape(-1)]

vertices.shape

### 2.2 - Vertex normals

We want to generate an array of vertex normals with the correct shape.


Calculate the normals.


In [ ]:
center = np.array([0, 0, 0])
normals = vertices - center[None, :]

Normalize the normals. Similarly to `vertices`, `normals` should have shape ($NM$,3).


In [ ]:
normals = normals / np.linalg.norm(normals, axis=1)[:, None]

normals.shape

### 2.3 - Triangles

We want to generate an array of triangles.


The triangulation is handled by SciPy's `ConvexHull` method. The format of the triangle array, `triangles`, is `[[v1, v2, v3], [v4, v5, v6], ...]`.


In [ ]:
triangles = ConvexHull(vertices).simplices

triangles.shape

### 2.4 - PyVista mesh

We want to create the PyVista mesh.


PyVista expects the triangles to be given in the format `[3, v1, v2, v3, 3, v4, v5, v6, ...]`. Reformat `triangles`.


In [ ]:
triangles_reformatted = np.zeros((triangles.shape[0], 4), dtype=int)
triangles_reformatted[:, 0] = 3
triangles_reformatted[:, 1:] = triangles

Flatten `triangles_reformatted`.


In [ ]:
triangles_flattened = triangles_reformatted.ravel()

triangles_flattened.shape

Create the PyVista mesh, and add the normals as point data. The mesh should have $NM$ vertices.


In [ ]:
cube_mesh = pv.PolyData(vertices, triangles_flattened)
cube_mesh.point_data["normals"] = normals

print(f"Number of vertices: {cube_mesh.n_points}")
print(f"Number of cells: {cube_mesh.n_cells}")

### 2.5 - Plot the mesh

We want to plot the PyVista mesh.


Plot the PyVista mesh, along with vertex normals.


In [ ]:
pl = pv.Plotter()

pl.add_mesh(cube_mesh, show_edges=True, color="lightblue")
arrows = cube_mesh.glyph(orient="normals", scale=False, factor=0.15)
pl.add_mesh(arrows, color="red")

pl.show()